<a href="https://colab.research.google.com/github/dinuwanfernando/BTCUSD_model/blob/main/BTCUSD_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
#import tensorflow as tf
#from tensorflow.keras import layers

In [53]:
#print('Num GPUs Available:', len(tf.config.experimental.list_physical_devices('GPU')))

In [54]:
#!nvidia-smi

In [55]:
#!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

In [56]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [57]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
import xgboost as xgb
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.api import VAR
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV
from math import sqrt
import matplotlib.pyplot as plt
!pip install prophet
from prophet import Prophet

In [58]:
%pip install oandapyV20
import oandapyV20
import oandapyV20.endpoints.instruments as instruments
from oandapyV20.contrib.requests import MarketOrderRequest
access_token = "25828916746768cb5cf6f0ec2b97218e-0b9185adf70ecd8cf1b53bb781a4b1f3"
%pip install git+https://github.com/yhilpisch/tpqoa.git
api = oandapyV20.API(access_token=access_token)

  Cloning https://github.com/yhilpisch/tpqoa.git to /tmp/pip-req-build-1b2i3ae6
  Running command git clone --filter=blob:none --quiet https://github.com/yhilpisch/tpqoa.git /tmp/pip-req-build-1b2i3ae6
  Resolved https://github.com/yhilpisch/tpqoa.git to commit 4a79f30b7095642844ef99741d9bfcb952e394db
  Preparing metadata (setup.py) ... done


In [59]:
import tpqoa
oanda = tpqoa.tpqoa("/content/drive/MyDrive/ML BTCUSD model/oanda.cfg.txt")

In [60]:
# Function to download historical data using Oanda API
def get_historical_data(instrument, start, end, granularity, price):
    oanda = tpqoa.tpqoa("/content/drive/MyDrive/ML BTCUSD model/oanda.cfg.txt")  # Update path if needed
    df = oanda.get_history(instrument=instrument, start=start, end=end, granularity=granularity, price=price)
    return df.dropna()


In [61]:
# Function to calculate technical indicators (replace with your selection)
def calculate_indicators(data):
    data['RSI'] = calculate_rsi(data['close'], window=14)  # Add your indicator functions here
    # Add more indicators like MACD, Bollinger Bands, etc.
    return data

In [91]:
def prepare_data(data):
    data['Date'] = pd.to_datetime(data.index)  # Assuming 'index' holds datetime data
    data.set_index('Date', inplace=True)  # Set 'Date' as index

    # Calculate technical indicators (replace with your selection)
    data['RSI'] = calculate_rsi(data['btc_close'], window=14)  # Update column name

    # Define features and target variable
    features = ['high', 'low', 'btc_volume', 'RSI']  # Adjust features based on indicators
    target = 'btc_close'  # Update target variable name
    return data[features], data[target]


In [63]:
date_var= '2024-04-01'

In [64]:
df_BTC_pp_01= oanda.get_history(instrument= 'BTC_USD', start= '2014-05-07', end='2016-05-06', granularity= 'H1', price='M')
df_BTC_pp_02= oanda.get_history(instrument= 'BTC_USD', start= '2016-05-07', end='2018-05-06', granularity= 'H1', price='M')
df_BTC_pp_03= oanda.get_history(instrument= 'BTC_USD', start= '2018-05-07', end='2020-05-06', granularity= 'H1', price='M')
df_BTC_pp_04= oanda.get_history(instrument= 'BTC_USD', start= '2020-05-07', end='2022-05-06', granularity= 'H1', price='M')
df_BTC_pp_05= oanda.get_history(instrument= 'BTC_USD', start= '2022-05-07', end='2023-05-06', granularity= 'H1', price='M')
df_BTC_pp_06= oanda.get_history(instrument= 'BTC_USD', start= '2023-05-07', end='2024-01-06', granularity= 'H1', price='M')
df_BTC_pp_07= oanda.get_history(instrument= 'BTC_USD', start= '2024-01-07', end=date_var, granularity= 'H1', price='M')

In [65]:
df_btc_p= pd.concat([df_BTC_pp_01,df_BTC_pp_02,df_BTC_pp_03,df_BTC_pp_04,df_BTC_pp_05,df_BTC_pp_06,
df_BTC_pp_07],axis=0).dropna()
df_btc_p.index= pd.to_datetime(df_btc_p.index)
df_btc_p= df_btc_p[df_btc_p.index < date_var].dropna()
###df_btc_dat.index= df_btc_data.index.date to add the time back
df_btc_p = df_btc_p.rename_axis('Date')
df_btc_p.rename(columns={'o': 'open', 'h': 'high', 'l': 'low', 'c': 'btc_close','volume':'btc_volume'}, inplace=True)
df_btc_p= df_btc_p[['high','low','btc_close','btc_volume']]
df_btc_p1=df_btc_p.astype(float)
print(df_btc_p1)
df_btc_p1.dtypes

                        high      low  btc_close  btc_volume
Date                                                        
2016-01-01 06:00:00    437.0    433.0      435.5      1616.0
2016-01-01 07:00:00    437.2    434.9      435.3      1620.0
2016-01-01 08:00:00    436.8    433.3      435.5      1377.0
2016-01-01 09:00:00    435.9    434.1      434.5       868.0
2016-01-01 10:00:00    435.9    433.4      433.9       789.0
...                      ...      ...        ...         ...
2024-03-29 19:00:00  69709.4  69400.0    69704.4     12317.0
2024-03-29 20:00:00  69757.8  69560.4    69640.6      9426.0
2024-03-31 21:00:00  70970.8  70645.2    70881.8      6836.0
2024-03-31 22:00:00  71120.0  70870.0    70983.4      8352.0
2024-03-31 23:00:00  71424.0  70944.8    71321.4     12109.0

[54869 rows x 4 columns]


high          float64
low           float64
btc_close     float64
btc_volume    float64
dtype: object

In [66]:
from sklearn.preprocessing import MinMaxScaler

# Feature Scaling (Optional)
scaler = MinMaxScaler(feature_range=(0, 1))  # Create a MinMaxScaler object


In [67]:
#Function to calculate rsi

def calculate_rsi(data, window=14):
    delta = data.diff()
    delta = delta[1:]  # Ignore NaN for first period
    up, down = delta.clip(lower=0), delta.clip(upper=0) * -1
    ema_up = up.ewm(alpha=1/window, min_periods=window, adjust=False).mean()
    ema_down = down.ewm(alpha=1/window, min_periods=window, adjust=False).mean()
    rs = ema_up / ema_down
    rsi = 100 - (1 / (1 + rs))
    return rsi


In [68]:
df_btc_p.columns

Index(['high', 'low', 'btc_close', 'btc_volume'], dtype='object')

In [69]:
df_btc_p.head()

,high,low,btc_close,btc_volume
Date,,,,
2016-01-01 06:00:00,437.0,433.0,435.5,1616
2016-01-01 07:00:00,437.2,434.9,435.3,1620
2016-01-01 08:00:00,436.8,433.3,435.5,1377
2016-01-01 09:00:00,435.9,434.1,434.5,868
2016-01-01 10:00:00,435.9,433.4,433.9,789


In [70]:
# Prepare data
features, target = prepare_data(df_btc_p.copy())
scaled_features = scaler.fit_transform(features)  # Apply scaling to features

In [71]:
# Split data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(scaled_features, target, test_size=0.2, random_state=42)


In [72]:
# Define and train XGBoost model (optional hyperparameter tuning can be added)
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.05)
xgb_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [79]:
# Define and train VAR model
#!pip install statsmodels
#from statsmodels.tsa.var import VAR
#var_model = VAR(X_train)
#var_model_fit = var_model.fit()

ModuleNotFoundError: No module named 'statsmodels.tsa.var'

In [80]:
# Define and train ARIMA model (replace with best fit ARIMA order after analysis)
arima_model = ARIMA(y_train, order=(5, 1, 1))  # Adjust order based on data analysis
arima_model_fit = arima_model.fit()

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsm

In [81]:
# Make predictions on testing set
xgb_pred = xgb_model.predict(X_test)
#var_pred = var_model_fit.forecast(steps=len(y_test))[:, 0]  # Select first column for single variable
arima_pred = arima_model_fit.predict(start=len(y_train), end=len(y_train) + len(y_test) - 1)

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [84]:
# Evaluate model performance
def evaluate_model(y_true, y_pred):
  mse = mean_squared_error(y_true, y_pred)
  mae = mean_absolute_error(y_true, y_pred)
  rmse = sqrt(mse)
  print('MSE:', mse)
  print('MAE:', mae)
  print('RMSE:', rmse)

evaluate_model(y_test, xgb_pred)
#evaluate_model(y_test, var_pred)
evaluate_model(y_test, arima_pred)

MSE: 38893.30746104006
MAE: 118.94324474136577
RMSE: 197.21386224360614
MSE: 298155920.7497322
MAE: 14234.850258462018
RMSE: 17267.19203430981


In [95]:
df_btc_p

,high,low,btc_close,btc_volume
Date,,,,
2016-01-01 06:00:00,437.0,433.0,435.5,1616
2016-01-01 07:00:00,437.2,434.9,435.3,1620
2016-01-01 08:00:00,436.8,433.3,435.5,1377
2016-01-01 09:00:00,435.9,434.1,434.5,868
2016-01-01 10:00:00,435.9,433.4,433.9,789
...,...,...,...,...
2024-03-29 19:00:00,69709.4,69400.0,69704.4,12317
2024-03-29 20:00:00,69757.8,69560.4,69640.6,9426
2024-03-31 21:00:00,70970.8,70645.2,70881.8,6836


In [96]:
future_dates.to_list()

[Timestamp('2024-04-01 23:00:00', freq='D'),
 Timestamp('2024-04-02 23:00:00', freq='D'),
 Timestamp('2024-04-03 23:00:00', freq='D'),
 Timestamp('2024-04-04 23:00:00', freq='D'),
 Timestamp('2024-04-05 23:00:00', freq='D')]

In [92]:
# Predict for future dates (adjust number of predictions as needed)
future_predictions = 5
future_dates = pd.date_range(start=df_btc_p.index[-1] + pd.DateOffset(days=1), periods=future_predictions)
future_features = scaler.transform(prepare_data(df_btc_p[future_dates.to_list()])[0])  # Assuming 'Datetime' is the index

# Make predictions using the best model (replace 'xgb_model' with your best model)
future_pred = xgb_model.predict(future_features)

# Optional: Visualize results (replace 'y_test' with actual values if desired)
plt.figure(figsize=(12, 6))
plt.plot(y_test.index, y_test, label='Actual')
plt.plot(y_test.index, xgb_pred, label='XGBoost Prediction')  # Replace with your best model's prediction
plt.legend()
plt.show()


NameError: name 'future_features' is not defined